# PowerShell Downgrade Attack
Detects PowerShell downgrade attack by comparing the host versions with the actually used engine version 2.0

## Rule Content
```
- title: PowerShell Downgrade Attack
  id: 6331d09b-4785-4c13-980f-f96661356249
  status: experimental
  description: Detects PowerShell downgrade attack by comparing the host versions
    with the actually used engine version 2.0
  references:
  - http://www.leeholmes.com/blog/2017/03/17/detecting-and-preventing-powershell-downgrade-attacks/
  tags:
  - attack.defense_evasion
  - attack.execution
  - attack.t1086
  author: Florian Roth (rule), Lee Holmes (idea)
  logsource:
    product: windows
    service: powershell-classic
    category: null
  detection:
    selection:
      EventID: 400
      EngineVersion: 2.*
    filter:
      HostVersion: 2.*
    condition: selection and not filter
  falsepositives:
  - Penetration Test
  - Unknown
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-powershell-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"400" AND powershell.engine.version.keyword:2.*) AND (NOT (powershell.host.version.keyword:2.*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()